# Privacy Preserving Training and Inference Machine Learning over Medical Data 

##  Flamby configuration - Download FedHeart

You need to download the FLamby dataset that we will use. For licensing reasons, these are not including directly in the FLamby installation.

To download the fed_heart dataset in `${FEDBIOMED_DIR}/data`, follow FLamby download instructions. In a nutshell:

execute on the **researcher** <br>
`source ${FEDBIOMED_DIR}/scripts/fedbiomed_environment researcher` <br> <br>
then execute on each **node** (where `${FEDBIOMED_DIR}` is the base directory of Fed-BioMed):<br>
`source ${FEDBIOMED_DIR}/scripts/fedbiomed_environment node`<br>
`pip install wget`<br>
`python ${FEDBIOMED_DIR}/docs/tutorial/concrete-ml/download.py --output-folder ${FEDBIOMED_DIR}/data`

In [76]:
import os
import torch 
from torch.utils.data import DataLoader
import numpy as np
from fedbiomed.common.training_plans import TorchTrainingPlan
from flamby.datasets.fed_heart_disease import FedHeartDisease
from fedbiomed.common.data import FlambyDataset, DataManager
from concrete.ml.torch.compile import compile_torch_model
from torch.nn.modules.loss import _Loss
import torch.nn as nn
from fedbiomed.researcher.environ import environ
tensorboard_dir = environ['TENSORBOARD_RESULTS_DIR']

In [86]:
TRAIN = True
FEDBIOMED_DIR = os.getenv('FEDBIOMED_DIR')
DATASET_TEST_PATH = f"{FEDBIOMED_DIR}/data"
MODEL_PATH = f"{FEDBIOMED_DIR}/docs/tutorials/concrete-ml/models/fed_sec_agg_model.pth"

## 1. Fed-BioMed

In [87]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [79]:
class FedHeartTrainingPlan(TorchTrainingPlan):
    
    class Baseline(nn.Module):
        
        def __init__(self):
            super().__init__()
            self.fc1 = nn.Linear(13, 16)
            self.fc2 = nn.Linear(16, 2)
            self.act = nn.LeakyReLU()
        def forward(self, x):
            x = self.act(self.fc1(x))
            x = self.fc2(x)
            return x
        
    class BaselineLoss(_Loss):
        def __init__(self):
            super().__init__()
            self.ce = torch.nn.CrossEntropyLoss()

        def forward(self, prediction: torch.Tensor, target: torch.Tensor):
            target = torch.squeeze(target, dim=1).type(torch.long)
            return self.ce(prediction, target)
    
    def init_model(self, model_args):
        return self.Baseline()

    def init_optimizer(self, optimizer_args):
        return torch.optim.AdamW(self.model().parameters(), lr=optimizer_args["lr"])

    def init_dependencies(self):
        return ["from flamby.datasets.fed_heart_disease import FedHeartDisease",
                "from torch.nn.modules.loss import _Loss",
                "from fedbiomed.common.data import FlambyDataset, DataManager"]

    def training_step(self, data, target):
        logits = self.model().forward(data)
        return self.BaselineLoss().forward(logits, target)

    def training_data(self, batch_size=2):
        dataset = FlambyDataset()
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset, **train_kwargs)

In [80]:
if TRAIN:
    batch_size=8
    num_updates=10
    def get_nb_max_rounds(num_updates, batch_size, num_epochs_pooled):
        return (486 // 4 // batch_size) * num_epochs_pooled // num_updates
    num_rounds = get_nb_max_rounds(num_updates=num_updates, batch_size=batch_size, num_epochs_pooled=50)
    num_rounds
    print(num_rounds)

75


In [81]:
if TRAIN:
    training_args = {
        'optimizer_args': {
            'lr': 5e-4,
        },
        'loader_args': {
            'batch_size': batch_size,
        },
        'num_updates': num_updates,
        'dry_run': False,
        'log_interval': 2,
        'test_ratio' : 0.0,
        'test_on_global_updates': False,
        'test_on_local_updates': False,
        'random_seed':42,
    }

    model_args = {}

## 2. Federated Learning Training with SecAgg

In [82]:
if TRAIN:
    from fedbiomed.researcher.experiment import Experiment
    from fedbiomed.researcher.aggregators.fedavg import FedAverage

    tags =  ['heart']

    exp_sec_agg = Experiment(tags=tags,
                     training_plan_class=FedHeartTrainingPlan,
                     training_args=training_args,
                     model_args=model_args,
                     round_limit=num_rounds,
                     aggregator=FedAverage(),
                     secagg=True,
                     tensorboard=True
                    )

2023-11-29 10:41:39,092 fedbiomed DEBUG - Node: node_7611fa0c-7efd-4374-8983-f00a33188e25 polling for the tasks
2023-11-29 10:41:39,094 fedbiomed DEBUG - Node: node_29db7d91-283f-4526-9897-7dffebeee47a polling for the tasks
2023-11-29 10:41:39,096 fedbiomed DEBUG - Node: node_5a149d86-47dc-491b-87c9-7656fb644c0a polling for the tasks
2023-11-29 10:41:39,097 fedbiomed DEBUG - Node: node_71bc7175-8f7e-483c-940f-f1cf88844771 polling for the tasks
2023-11-29 10:41:39,101 fedbiomed INFO - Node selected for training -> node_7611fa0c-7efd-4374-8983-f00a33188e25
2023-11-29 10:41:39,101 fedbiomed INFO - Node selected for training -> node_29db7d91-283f-4526-9897-7dffebeee47a
2023-11-29 10:41:39,102 fedbiomed INFO - Node selected for training -> node_71bc7175-8f7e-483c-940f-f1cf88844771
2023-11-29 10:41:39,103 fedbiomed INFO - Node selected for training -> node_5a149d86-47dc-491b-87c9-7656fb644c0a
2023-11-29 10:41:39,106 fedbiomed DEBUG - Model file has been saved: /workspaces/Projects/fedbiomed/

In [83]:
tensorboard --logdir "$tensorboard_dir"

Reusing TensorBoard on port 6006 (pid 19426), started 0:30:16 ago. (Use '!kill 19426' to kill it.)

In [84]:
if TRAIN:
    exp_sec_agg.run()

2023-11-29 10:41:42,819 fedbiomed DEBUG - Secagg context for default_biprime0 is already existing on researcher researcher_id='researcher_39937477-12dd-4660-8690-8d0fd2a265be'
2023-11-29 10:41:42,824 fedbiomed DEBUG - Node: node_29db7d91-283f-4526-9897-7dffebeee47a polling for the tasks
2023-11-29 10:41:42,826 fedbiomed DEBUG - Node: node_71bc7175-8f7e-483c-940f-f1cf88844771 polling for the tasks
2023-11-29 10:41:42,827 fedbiomed DEBUG - Node: node_7611fa0c-7efd-4374-8983-f00a33188e25 polling for the tasks
2023-11-29 10:41:42,828 fedbiomed DEBUG - Node: node_5a149d86-47dc-491b-87c9-7656fb644c0a polling for the tasks
2023-11-29 10:41:42,836 fedbiomed DEBUG - Node: node_7611fa0c-7efd-4374-8983-f00a33188e25 polling for the tasks
2023-11-29 10:41:42,843 fedbiomed DEBUG - Node: node_29db7d91-283f-4526-9897-7dffebeee47a polling for the tasks
2023-11-29 10:41:42,844 fedbiomed DEBUG - Node: node_5a149d86-47dc-491b-87c9-7656fb644c0a polling for the tasks
2023-11-29 10:41:42,845 fedbiomed DEBUG 

2023-11-29 10:41:46,638 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7611fa0c-7efd-4374-8983-f00a33188e25 
					 Round 1 | Iteration: 8/10 (80%) | Samples: 16/80
 					 Loss: 1.099793 
					 ---------
2023-11-29 10:41:46,657 fedbiomed INFO - TRAINING 
					 NODE_ID: node_71bc7175-8f7e-483c-940f-f1cf88844771 
					 Round 1 | Iteration: 10/10 (100%) | Samples: 20/80
 					 Loss: 0.716280 
					 ---------
2023-11-29 10:41:46,661 fedbiomed INFO - INFO
					 NODE node_71bc7175-8f7e-483c-940f-f1cf88844771
					 MESSAGE: Encrypting model parameters. This process can take some time depending on model size.
-----------------------------------------------------------------
2023-11-29 10:41:46,666 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7611fa0c-7efd-4374-8983-f00a33188e25 
					 Round 1 | Iteration: 10/10 (100%) | Samples: 20/80
 					 Loss: 1.010768 
					 ---------
2023-11-29 10:41:46,673 fedbiomed INFO - INFO
					 NODE node_7611fa0c-7efd-4374-8983-f00a33188e25
					 MESSAGE: Encrypt

2023-11-29 10:41:47,119 fedbiomed INFO - TRAINING 
					 NODE_ID: node_29db7d91-283f-4526-9897-7dffebeee47a 
					 Round 2 | Iteration: 4/10 (40%) | Samples: 8/80
 					 Loss: 0.986280 
					 ---------
2023-11-29 10:41:47,122 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5a149d86-47dc-491b-87c9-7656fb644c0a 
					 Round 2 | Iteration: 8/10 (80%) | Samples: 16/80
 					 Loss: 0.415924 
					 ---------
2023-11-29 10:41:47,125 fedbiomed INFO - TRAINING 
					 NODE_ID: node_71bc7175-8f7e-483c-940f-f1cf88844771 
					 Round 2 | Iteration: 1/10 (10%) | Samples: 2/80
 					 Loss: 1.077011 
					 ---------
2023-11-29 10:41:47,147 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5a149d86-47dc-491b-87c9-7656fb644c0a 
					 Round 2 | Iteration: 10/10 (100%) | Samples: 20/80
 					 Loss: 0.581552 
					 ---------
2023-11-29 10:41:47,156 fedbiomed INFO - INFO
					 NODE node_5a149d86-47dc-491b-87c9-7656fb644c0a
					 MESSAGE: Encrypting model parameters. This process can take some time depending on 

2023-11-29 10:41:47,563 fedbiomed INFO - TRAINING 
					 NODE_ID: node_29db7d91-283f-4526-9897-7dffebeee47a 
					 Round 3 | Iteration: 1/10 (10%) | Samples: 2/80
 					 Loss: 1.005523 
					 ---------
2023-11-29 10:41:47,578 fedbiomed INFO - TRAINING 
					 NODE_ID: node_29db7d91-283f-4526-9897-7dffebeee47a 
					 Round 3 | Iteration: 2/10 (20%) | Samples: 4/80
 					 Loss: 1.072324 
					 ---------
2023-11-29 10:41:47,595 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5a149d86-47dc-491b-87c9-7656fb644c0a 
					 Round 3 | Iteration: 1/10 (10%) | Samples: 2/80
 					 Loss: 1.044842 
					 ---------
2023-11-29 10:41:47,615 fedbiomed INFO - TRAINING 
					 NODE_ID: node_29db7d91-283f-4526-9897-7dffebeee47a 
					 Round 3 | Iteration: 4/10 (40%) | Samples: 8/80
 					 Loss: 0.983729 
					 ---------
2023-11-29 10:41:47,619 fedbiomed INFO - TRAINING 
					 NODE_ID: node_71bc7175-8f7e-483c-940f-f1cf88844771 
					 Round 3 | Iteration: 1/10 (10%) | Samples: 2/80
 					 Loss: 1.054052 
					 

2023-11-29 10:41:47,930 fedbiomed INFO - Saved aggregated params for round 2 in /workspaces/Projects/fedbiomed/var/experiments/Experiment_0012/aggregated_params_fb8862c3-0949-47ec-bc1e-4f065d51e047.mpk
2023-11-29 10:41:47,930 fedbiomed INFO - Sampled nodes in round 3 ['node_7611fa0c-7efd-4374-8983-f00a33188e25', 'node_29db7d91-283f-4526-9897-7dffebeee47a', 'node_71bc7175-8f7e-483c-940f-f1cf88844771', 'node_5a149d86-47dc-491b-87c9-7656fb644c0a']
2023-11-29 10:41:47,932 fedbiomed INFO - Sending request 
					 To: node_7611fa0c-7efd-4374-8983-f00a33188e25 
					 Request: : TRAIN
 -----------------------------------------------------------------
2023-11-29 10:41:47,933 fedbiomed INFO - Sending request 
					 To: node_29db7d91-283f-4526-9897-7dffebeee47a 
					 Request: : TRAIN
 -----------------------------------------------------------------
2023-11-29 10:41:47,933 fedbiomed INFO - Sending request 
					 To: node_71bc7175-8f7e-483c-940f-f1cf88844771 
					 Request: : TRAIN
 --------------

...

2023-11-29 10:42:31,011 fedbiomed INFO - INFO
					 NODE node_29db7d91-283f-4526-9897-7dffebeee47a
					 MESSAGE: Encrypting model parameters. This process can take some time depending on model size.
-----------------------------------------------------------------
2023-11-29 10:42:31,021 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5a149d86-47dc-491b-87c9-7656fb644c0a 
					 Round 72 | Iteration: 6/10 (60%) | Samples: 12/80
 					 Loss: 0.342828 
					 ---------
2023-11-29 10:42:31,034 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7611fa0c-7efd-4374-8983-f00a33188e25 
					 Round 72 | Iteration: 4/10 (40%) | Samples: 8/80
 					 Loss: 0.065103 
					 ---------
2023-11-29 10:42:31,038 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5a149d86-47dc-491b-87c9-7656fb644c0a 
					 Round 72 | Iteration: 8/10 (80%) | Samples: 16/80
 					 Loss: 0.249303 
					 ---------
2023-11-29 10:42:31,062 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5a149d86-47dc-491b-87c9-7656fb644c0a 
					 Round 72

2023-11-29 10:42:31,586 fedbiomed INFO - TRAINING 
					 NODE_ID: node_29db7d91-283f-4526-9897-7dffebeee47a 
					 Round 73 | Iteration: 1/10 (10%) | Samples: 2/80
 					 Loss: 0.391378 
					 ---------
2023-11-29 10:42:31,599 fedbiomed INFO - TRAINING 
					 NODE_ID: node_71bc7175-8f7e-483c-940f-f1cf88844771 
					 Round 73 | Iteration: 6/10 (60%) | Samples: 12/80
 					 Loss: 0.226278 
					 ---------
2023-11-29 10:42:31,604 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7611fa0c-7efd-4374-8983-f00a33188e25 
					 Round 73 | Iteration: 1/10 (10%) | Samples: 2/80
 					 Loss: 0.508251 
					 ---------
2023-11-29 10:42:31,613 fedbiomed INFO - TRAINING 
					 NODE_ID: node_29db7d91-283f-4526-9897-7dffebeee47a 
					 Round 73 | Iteration: 2/10 (20%) | Samples: 4/80
 					 Loss: 1.555840 
					 ---------
2023-11-29 10:42:31,617 fedbiomed INFO - TRAINING 
					 NODE_ID: node_71bc7175-8f7e-483c-940f-f1cf88844771 
					 Round 73 | Iteration: 8/10 (80%) | Samples: 16/80
 					 Loss: 1.097633 

2023-11-29 10:42:31,953 fedbiomed DEBUG - Node: node_7611fa0c-7efd-4374-8983-f00a33188e25 polling for the tasks
2023-11-29 10:42:31,960 fedbiomed DEBUG - Node: node_71bc7175-8f7e-483c-940f-f1cf88844771 polling for the tasks
2023-11-29 10:42:31,962 fedbiomed DEBUG - Node: node_5a149d86-47dc-491b-87c9-7656fb644c0a polling for the tasks
2023-11-29 10:42:31,963 fedbiomed DEBUG - Node: node_29db7d91-283f-4526-9897-7dffebeee47a polling for the tasks
2023-11-29 10:42:32,124 fedbiomed INFO - TRAINING 
					 NODE_ID: node_29db7d91-283f-4526-9897-7dffebeee47a 
					 Round 74 | Iteration: 1/10 (10%) | Samples: 2/80
 					 Loss: 0.383726 
					 ---------
2023-11-29 10:42:32,132 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5a149d86-47dc-491b-87c9-7656fb644c0a 
					 Round 74 | Iteration: 1/10 (10%) | Samples: 2/80
 					 Loss: 0.021903 
					 ---------
2023-11-29 10:42:32,134 fedbiomed INFO - TRAINING 
					 NODE_ID: node_5a149d86-47dc-491b-87c9-7656fb644c0a 
					 Round 74 | Iteration: 2/10 (20%

2023-11-29 10:42:32,495 fedbiomed DEBUG - Aggregation is completed in 0.01 seconds.
2023-11-29 10:42:32,497 fedbiomed INFO - Validation is completed.
2023-11-29 10:42:32,497 fedbiomed INFO - Aggregating encrypted parameters. This process may take some time depending on model size.
2023-11-29 10:42:32,560 fedbiomed INFO - Aggregating 4 parameters from 4 nodes.
2023-11-29 10:42:32,561 fedbiomed DEBUG - Aggregation is completed in 0.06 seconds.
2023-11-29 10:42:32,563 fedbiomed INFO - Saved aggregated params for round 73 in /workspaces/Projects/fedbiomed/var/experiments/Experiment_0012/aggregated_params_0da02f61-68fb-47bb-bbdf-dc91e5ec551d.mpk
2023-11-29 10:42:32,564 fedbiomed INFO - Sampled nodes in round 74 ['node_7611fa0c-7efd-4374-8983-f00a33188e25', 'node_29db7d91-283f-4526-9897-7dffebeee47a', 'node_71bc7175-8f7e-483c-940f-f1cf88844771', 'node_5a149d86-47dc-491b-87c9-7656fb644c0a']
2023-11-29 10:42:32,566 fedbiomed INFO - Sending request 
					 To: node_7611fa0c-7efd-4374-8983-f00a33

2023-11-29 10:42:33,022 fedbiomed INFO - INFO
					 NODE node_7611fa0c-7efd-4374-8983-f00a33188e25
					 MESSAGE: Encrypting model parameters. This process can take some time depending on model size.
-----------------------------------------------------------------
2023-11-29 10:42:33,035 fedbiomed INFO - TRAINING 
					 NODE_ID: node_71bc7175-8f7e-483c-940f-f1cf88844771 
					 Round 75 | Iteration: 10/10 (100%) | Samples: 20/80
 					 Loss: 0.283895 
					 ---------
2023-11-29 10:42:33,039 fedbiomed INFO - INFO
					 NODE node_71bc7175-8f7e-483c-940f-f1cf88844771
					 MESSAGE: Encrypting model parameters. This process can take some time depending on model size.
-----------------------------------------------------------------
2023-11-29 10:42:33,078 fedbiomed INFO - INFO
					 NODE node_7611fa0c-7efd-4374-8983-f00a33188e25
					 MESSAGE: Encryption is completed!
-----------------------------------------------------------------
2023-11-29 10:42:33,085 fedbiomed INFO - INFO
					 NODE 

In [88]:
# save the last fl model
if TRAIN:
    torch.save(exp_sec_agg.aggregated_params()[num_rounds-1]['params'],MODEL_PATH)

# 3. Inference

In [89]:
fed_sec_agg_model = FedHeartTrainingPlan().Baseline()
model_state_dict = torch.load(MODEL_PATH)
fed_sec_agg_model.load_state_dict(model_state_dict)
fed_sec_agg_model.eval()

Baseline(
  (fc1): Linear(in_features=13, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=2, bias=True)
  (act): LeakyReLU(negative_slope=0.01)
)

In [90]:
test_dataset = FedHeartDisease(center=0,pooled=True, train=False, data_path=DATASET_TEST_PATH)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

## 3.1. Inference with torch

In [91]:
def test_torch(net, test_loader):
    """Test the network: measure accuracy on the test set."""

    # Freeze normalization layers
    net.eval()

    all_y_pred = np.zeros((len(test_loader)), dtype=np.int64)
    all_targets = np.zeros((len(test_loader)), dtype=np.int64)

    # Iterate over the batches
    idx = 0
    for data, target in test_loader:
        # Accumulate the ground truth labels
        endidx = idx + target.shape[0]
        all_targets[idx:endidx] = target.numpy()

        # Run forward and get the predicted class id
        logits = torch.sigmoid(net(data))
        output = logits.argmax(1).detach().numpy()
        all_y_pred[idx:endidx] = output

        idx += target.shape[0]

    # Print out the accuracy as a percentage
    n_correct = np.sum(all_targets == all_y_pred)
    print(
        f"Test accuracy over plaintext model: "
        f"{n_correct / len(test_loader) * 100:.2f}%"
    )


In [92]:
test_torch(fed_sec_agg_model, test_dataloader)

Test accuracy over plaintext model: 77.56%


## 3.2. Inference with Concrete-ML

In [93]:
def test_with_concrete(quantized_module, test_loader, use_sim):
    """Test a neural network that is quantized and compiled with Concrete ML."""

    # Casting the inputs into int64 is recommended
    all_y_pred = np.zeros((len(test_loader)), dtype=np.int64)
    all_targets = np.zeros((len(test_loader)), dtype=np.int64)

    # Iterate over the test batches and accumulate predictions and ground truth labels in a vector
    idx = 0
    for data, target in test_loader:
        data = data.numpy()
        target = target.numpy()

        fhe_mode = "simulate" if use_sim else "execute"

        # Quantize the inputs and cast to appropriate data type
        logits = torch.tensor(quantized_module.forward(data, fhe=fhe_mode), requires_grad=False)

        endidx = idx + target.shape[0]

        all_targets[idx:endidx] = target

        # Get the predicted class id and accumulate the predictions
        y_pred = torch.sigmoid(logits).argmax(1).numpy()
        all_y_pred[idx:endidx] = y_pred

        # Update the index
        idx += target.shape[0]
    n_correct = np.sum(all_targets == all_y_pred)
    print(
        f"Test accuracy over encrypted model: "
        f"{n_correct / len(test_loader) * 100:.2f}%"
    )


In [98]:
# concrete ml is using the traceback, 
# while fed-biomed for logging reasons fixs it to 3, to use concrete-ml we reset to the default value
import sys
sys.tracebacklimit = 1000

In [99]:
n_bits = 6
compile_set = np.random.randint(0, 10, (100, 13)).astype(float)
q_module = compile_torch_model(fed_sec_agg_model, compile_set, rounding_threshold_bits=6)
test_with_concrete(q_module, test_dataloader, True)

Test accuracy over encrypted model: 77.56%
